In [1]:
import networkx as nx
%matplotlib inline
from random import randrange as rr

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

init_notebook_mode(connected=True)

In [2]:
df = pd.read_csv("data/wtf.csv")
df['sitc4'] = df['sitc4'].astype(str)
del df['Unnamed: 0']

gdf = df.groupby(["sitc4","exporter"]).sum().reset_index()
gdf = gdf[['sitc4','exporter','value']]

rca_df = pd.read_csv("data/rca.csv")
del rca_df['Unnamed: 0']
rca_df = rca_df.dropna(subset=['sitc4', 'exporter'])

In [3]:
products = list(rca_df.sitc4.unique())
countries = list(rca_df.exporter.unique())

In [4]:
def model_cap(N_a,r,q):
    # Create Country Capabilities Matrix
    C_ca=np.zeros((len(countries),N_a))

    for i in range(len(countries)):
        for j in range(N_a):
            C_ca[i][j]=np.random.choice([0,1],p =[1-r, r]) 

    # Create Product Required Capabilities
    P_pa=np.zeros((len(products),N_a))

    for i in range(len(products)):
        for j in range(N_a):
            P_pa[i][j]=np.random.choice([0,1],p =[1-q, q]) 

    # Create Mcp based on C and P

    M_cp=np.zeros((len(countries),len(products)))

    for i in range(len(countries)):
        for j in range(len(products)):
            sum_r = 0
            sum_l = 0
            for k in range(N_a):
                sum_r += P_pa.item((j, k))*C_ca.item((i, k))
            for k in range(N_a):
                sum_l += P_pa.item((j, k))

            if sum_l == sum_r:
                M_cp[i][j] = 1 

    # Creation k dictionary

    k = dict()
    for c in range(len(countries)):
        country=countries[c]
        sum_p = 0
        for i in range(len(products)):
            sum_p += M_cp.item((c, i))
        k[country] = []
        k[country].append(sum_p)

    for p in range(len(products)):
        product=products[p]
        sum_c = 0
        for i in range(len(countries)):
            sum_c += M_cp.item((i, p))
        k[product] = []
        k[product].append(sum_c)

    for c in range(len(countries)):
        country=countries[c]
        sum_p = 0
        for p in range(len(products)):
            sum_p += M_cp.item((c, p))*k[products[p]][0]
        sum_p = sum_p/k[country][0]
        k[country].append(sum_p)

    kdf = pd.DataFrame({'country_name': countries,
                        'kc0': list(map(lambda x: k[x][0],countries)),
                    'kc1': list(map(lambda x: k[x][1],countries))})
    return kdf



In [5]:
kdf1 = model_cap(N_a=200,r=0.7,q=0.05)
kdf2 = model_cap(N_a=200,r=0.55,q=0.05)
kdf3 = model_cap(N_a=50,r=0.7,q=0.1)
kdf4 = model_cap(N_a=50,r=0.55,q=0.1)

In [8]:
fig = make_subplots(rows=2, cols=2, start_cell="bottom-left")





fig.add_trace(go.Scatter(x=kdf1['kc0'],
                  y=kdf1['kc1'],
                  text=kdf1['country_name'],
                  mode='markers',
                  marker=dict(color='rgba(48, 217, 189, 1)'),
                  name='Diagram kdf1'),
              row=1, col=1)

fig.add_trace(go.Scatter(x=kdf2['kc0'],
                  y=kdf2['kc1'],
                  text=kdf2['country_name'],
                  mode='markers',
                  marker=dict(color='rgba(48, 217, 189, 1)'),
                  name='Diagram kdf2'),
              row=1, col=2)

fig.add_trace(go.Scatter(x=kdf3['kc0'],
                  y=kdf3['kc1'],
                  text=kdf3['country_name'],
                  mode='markers',
                  marker=dict(color='rgba(48, 217, 189, 1)'),
                  name='Diagram kdf3'),
              row=2, col=1)

fig.add_trace(go.Scatter(x=kdf4['kc0'],
                  y=kdf4['kc1'],
                  text=kdf4['country_name'],
                  mode='markers',
                  marker=dict(color='rgba(48, 217, 189, 1)'),
                  name='Diagram kdf4'),
              row=2, col=2)



# Update xaxis properties
fig.update_xaxes(title_text="$k_{c,0}$", row=1, col=1)
fig.update_xaxes(title_text="$k_{c,0}$", row=1, col=2)
fig.update_xaxes(title_text="$k_{c,0}$", row=2, col=1)
fig.update_xaxes(title_text="$k_{c,0}$", row=2, col=2)

# Update yaxis properties
fig.update_yaxes(title_text="$k_{c,1}$", row=1, col=1)
fig.update_yaxes(title_text="", row=1, col=2)
fig.update_yaxes(title_text="$k_{c,1}$", row=2, col=1)
fig.update_yaxes(title_text="", row=2, col=2)

# Update title and height
fig.update_layout(title_text="Diagrams", height=700)



fig.show()